# Signall Processing工具使用

在理論課程中已有補充數位信號處裡的基礎與常見分析方式。

這邊我們介紹用python完成這些訊號處裡的一些工具，方便後面AI modeling時使用。

我們主要會聚焦在聲音處裡上，不過其中很多概念與工具都可以用在其他訊號的處理。

課程包含以下內容:
* Audio data
* Up/Down Sampling
* Fast Fourier Transform (FFT)
* Short-Time Fourier Transform (STFT)
* Mel-Spectrogram
* Mel-Frequency Cepstral Coefficient (MFCC)

In [ ]:
# 處理音訊最常用librosa
!pip install librosa

In [ ]:
import librosa
import IPython.display as idp  # 播音工具
import librosa.display as ldp  # 畫頻譜圖工具
import numpy as np  # 輔助運算
import matplotlib.pyplot as plt  # 輔助畫圖
from plotly import express as px

In [ ]:
def plot_signal(signal, sr, start=0, end=None, labels=None, title=None):
    # Visualizes signal data
    # Args:
    #  signal (list of array of int) - 時間點對應的訊號，列表內時間序列數量為D，每筆資料長度為T，若非為列表則轉為列表
    #  start (int) - 開始的資料序(第幾筆)
    #  end (int) -   結束繪製的資料序(第幾筆)
    #  labels (list of strings)- 對於多時間序列或多維度的標註
    #  title (string)- 圖片標題

    # 若資料只有一筆，則轉為list
    if type(signal) != list:
        signal = [signal]

    if not end:
        end = len(signal[0])
    time = (np.arange(len(signal[0]))/sr)[start:end]
    if labels:
        # 設立dictionary, 讓plotly畫訊號線時可以標註label
        dictionary = {"time": time}
        for idx, l in enumerate(labels):
            # 截斷資料，保留想看的部分，並分段紀錄於dictionary中
            dictionary.update({l: signal[idx][start:end]})
        # 畫訊號線
        fig = px.line(dictionary,
                      x="time",
                      y=list(dictionary.keys())[1:],
                      width=1000,
                      height=400,
                      title=title)
    else:
        # 畫訊號線
        fig = px.line(x=time,
                      y=signal,
                      width=1000,
                      height=400,
                      title=title)
    fig.show()

## Audio Data

我們可以使用Librosa讀取資料，這邊提供範例資料:
* google小姐的一段語音 (以這個為示範) - data/mrs_google_aia.mp3
* 鋼琴聲(Mozart, Sonata K.331 - I. Andante grazioso) (可以拿來自行練習) - data/signal_googl.wav

In [ ]:
# 下載檔案並存到data資料夾
!mkdir -p data
!wget -O data/demo.zip https://github.com/TA-aiacademy/course_3.0/releases/download/TSRNN/sound_demo.zip
!unzip data/demo.zip -d data

In [ ]:
# 使用librosa.load可以讀取檔案，讀出來預設是float32格式
#  'path' - 檔名
#  'sr' - 可以指定聲音檔讀出來後的sampling rate

googl_, sr = librosa.load("data/mrs_google_aia.mp3")  # 讀出訊號檔以及sampling rate
piano_, sr = librosa.load("data/signal_piano.wav")  # 讀出訊號檔以及sampling rate

In [ ]:
# 使用IPython的display.Audio可以放出來試聽
idp.Audio("data/signal_piano.wav")

In [ ]:
idp.Audio("data/mrs_google_aia.mp3")

In [ ]:
%matplotlib inline
# 使用librosa的display.waveplot可以將波形的波封畫出來，並且對上實際時間做為參考
ldp.waveshow(googl_, sr=sr)

In [ ]:
# librosa那邊的圖不太能操控或縮放，所以也可以自己用plotly寫的比較好操控，可以Zoom-in
plot_signal(googl_, sr, labels=['google'])

把訊號畫出來對解析很重要，但每種情況可能需求都不太一樣

例如音訊可能十分密集，所以可能可以不需要看得太細，可以稍微re-sample一下

## Re-Sampling

Sampling rate的調整是處理訊號重要的一環，要是要將兩個訊號疊加，但發現兩者雖然sample數相同sampling rage卻不相符則會有錯誤的疊加效果。

Re- sampling有各種方法，對訊號皆有擾動，詳細可參考librosa官網參數```res_type```:
https://librosa.org/doc/main/generated/librosa.resample.html

預設的就一般訊號分析來講很夠用了。

### Downsample

In [ ]:
# Downsample 減少sampling rate,也減少資料點數
googl_3000 = librosa.resample(googl_, orig_sr=sr, target_sr=3000)

In [ ]:
# 同樣是10秒錄音，在resample後點數會變少
print(len(googl_), len(googl_3000))
# 若是需求的sampling rate剛好是原本的因數，那也可以直接依downsample的倍率做sample
googl_11025 = googl_[::2]
print(len(googl_), len(googl_11025))

In [ ]:
# 可聽出聲音變得比較朦朧，第一是downsample後失真，第二也是人類對特定頻段較有知覺，剛好有些頻段被截掉了
# 參考 https://en.wikipedia.org/wiki/Equal-loudness_contour
idp.Audio(googl_3000, rate=3000)

In [ ]:
# 從waveplot上可以看到在經過downsample後 有點失真了
ldp.waveshow(googl_[:int(0.2*sr)], sr=sr)
ldp.waveshow(googl_3000[:int(0.2*3000)], sr=3000)

### Upsample

In [ ]:
# Upsample 增加sampling rate,也增加資料點數
googl_44100 = librosa.resample(googl_, orig_sr=sr, target_sr=44100)

In [ ]:
# 可看一下資料數
print(len(googl_), len(googl_44100))

In [ ]:
# 在upsample時不會有失真的情況，因為是類似在資料間插值補值
ldp.waveshow(googl_44100[:int(0.2*44100)], sr=44100)
ldp.waveshow(googl_[:int(0.2*sr)], sr=sr)

一般來講upsampling都不存在標準答案，因為就是sampling rate不夠才要補。

Upsampling本身後來在AI領域也變成一個議題，叫Super Resolution，使用訓練好的神經網路模型來做upsampling增加解析度。

## Fast Fourier Transform

分析訊號的成分最常見的方式是頻譜分析，連續訊號可以藉由Fourier Transform得到頻譜，也就是訊號在頻率上的分布。

而對於數位訊號，我們可以藉由Discrete Fourier Transform得到頻譜
$$𝐗_k≔\sum\limits_{𝑛=0}^{𝑁−1}𝑥_𝑛 𝑒^{(−𝑗2𝜋𝑘𝑛/𝑁)}$$

<img src=https://i.imgur.com/CZh8cYU.png width=800>


Scipy或者其他套件包有提供一些方式做快速的Discrete Fourier Transform，稱為Fast Fourier Transform(FFT)。

若是訊號分析則用Scipy就好，若是要整合到神經網路上可能就得使用Tensorflow或Pytorch內建的fft。

In [ ]:
from scipy import fft

$$$$這邊因為我們做的是離散的Fourier轉換，若要看到原本單位(seconds, Hz)則需進行轉換。
* 時間: $n=f_s t$
* 頻率: $k=f N /f_s$

N為參與FFT的點數，$f_s$則是sampling rate。

In [ ]:
# 使用scipy.fft.fft可以對訊號做fft
#  'x' - 資料
#  'n' - FFT點數，通常不指定，預設為資料總點數
# 記得做完Fourier Transform後m, 出來都是複數 (預設為complex64格式)
# (不管是continuous/discrete/fast Fourier Transfor)
N = len(googl_)
googl_f = fft.fft(googl_[:N])
frequency = np.linspace(0, sr, N)
max_f = sr/2  # 設定想看到的最大頻率 (Hz)
max_k = int(max_f*N/sr)  # 轉成k

# 通常我們是看這個複數的magnitude, 取absolute就可以做到
plt.plot(frequency[:max_k], abs(googl_f[:max_k]))
plt.xlabel("frequency(Hz)")

在頻譜圖中我們可以看到較為高峰的點就表示訊號有較多該頻率成分。

我們可以試著把做頻譜的時間縮點一點，來看看最前面一段的頻譜

In [ ]:
N1 = int(0.12*sr)
N2 = int(0.15*sr)
plt.plot(np.arange(N1, N2)/sr, googl_[N1:N2])
plt.xlabel("time(s)")
plt.show()
googl_f2 = fft.fft(googl_[N1:N2])
frequency2 = np.linspace(0, sr, N2-N1)
max_f = 4000  # 設定想看到的最大頻率 (Hz)
max_k2 = int(max_f*(N2-N1)/sr)  # 轉成k
plt.plot(frequency2[:max_k2], abs(googl_f2[:max_k2]))
plt.xlabel("frequency(Hz)")

我們取第一個音的頻譜來看可看到一個主頻率，就語音來講會比較模糊一點

In [ ]:
plt.plot(frequency[:max_k], abs(googl_f[:max_k]))
plt.plot(frequency2[:max_k2], abs(googl_f2[:max_k2])*200)
plt.xlabel("frequency(Hz)")

擷取與整段對比:
* 整段時長較長，堆積出的能量比較高
* 整段音調混雜，彼此交錯

亦可用```librosa.power_to_db```轉換成分貝頻譜來看，會使scale差比較多的部分較為緩和，方便比較

In [ ]:
librosa.power_to_db(abs(googl_f2))
plt.plot(frequency[:max_k], librosa.power_to_db(abs(googl_f[:max_k])))
plt.plot(frequency2[:max_k2], librosa.power_to_db(abs(googl_f2[:max_k2])))
plt.xlabel("frequency(Hz)")

## Short-Time Fourier Transform

若是想統計出一段內出現哪些音符，則這個FFT就夠用了，但通常想知道的是在每個時間點頻率長什麼樣子就必須要做時頻分析。

時頻分析中最常見的方式就是Sort-Time Fourier Transform:
1. Windowing
2. 個別做頻譜

公式:

$𝑋[q,k] = \sum\limits_{𝑛^′=⌈−𝑁/2⌉}^{⌈𝑁/2⌉−1}𝑥[𝑛^′+𝑞𝐻]𝑤[𝑛^′] 𝑒^\frac{−𝑗2𝜋𝑘𝑛^′}{𝑁}$

* q- 每個window的離散時間點
* k- 離散頻率點
* n'- 原訊號的離散時間點
* N- window內FFT點數，同時是window size
* H- hop size，每個window離多少n'
* w- windowing function
* x- signal

<img src=https://i.imgur.com/Ji4xT6o.png width = 600>


Librosa裡面沒有FFT但有很多時頻分析的工具。

我們這邊使用```librosa.stft```來做時頻分析

In [ ]:
# 使用librosa.stft可以對訊號做stft
#  'y' - 資料
#  'n_fft' - FFT點數，同時是window長度，這邊就一定要指定了，預設2048，可根據自己想看到的頻率範圍調整
#  'hop_length' - 每個window間要跳多長
#  'window' - windowing function，預設是 'hann'
# 出來是複數 (預設為complex64格式)

N = 2048
H = 512
googl_S = librosa.stft(googl_, n_fft=N, hop_length=H)

In [ ]:
# 可以看出，我們使用librosa STFT可生出一個K x Q 的矩陣
print(googl_S.shape)
print(type(googl_S))
print(type(googl_S[0, 0]))

In [ ]:
# K的大小來自N points/2，因為大於sampling rate/2的訊號會aliase
print(N//2+1)
# Q的大小來自總長度除以hop length，就是windowing的次數
print(len(googl_)//H+1)

使用```librosa.display.specshow```可以畫出頻譜圖

In [ ]:
import librosa.display as ldp  # 顯示訊號工具

In [ ]:
# 使用librosa.display.specshow畫出Spectrogram
#  'data' - Spectrogram
#  'sr' - sampling rate
#  'x_axis' - x 軸刻度單位
#  'y_axis' - y 軸刻度單位，預設為 'hz'，若要使用log scale可以用 'log'
#  'cmap' - color map

plt.figure(figsize=(6, 5))
ldp.specshow(abs(googl_S),
             sr=sr,
             x_axis="s",
             y_axis="hz",
             cmap="jet")
plt.colorbar(format="%+4.f")
plt.show()

若全圖檢視可以看到所有包含的頻率，但其中Y軸的scale有點太大使得較低頻較有資訊的部分看不見，可以轉而使用log scale frequency

In [ ]:
plt.figure(figsize=(6, 5))
ldp.specshow(googl_S, sr=sr, x_axis="s", y_axis="log", cmap="jet")  # **
plt.colorbar(format="%+4.f")
plt.clim([5, 98])  # **
plt.show()

結果如果對比不夠明顯，可以使用```librosa.power_to_db```轉成分貝來看

In [ ]:
googl_S_db = librosa.power_to_db(abs(googl_S))  # **
plt.figure(figsize=(6, 5))
ldp.specshow(googl_S_db, sr=sr, x_axis="s", y_axis="log", cmap="jet")
plt.colorbar(format="%+4.f")
plt.show()

如果對於power較小的部分不想看到，可以調整```clim```

In [ ]:
googl_S_db = librosa.power_to_db(abs(googl_S))
plt.figure(figsize=(6, 5))
ldp.specshow(googl_S_db, sr=sr, x_axis="s", y_axis="log", cmap="jet")
plt.colorbar(format="%+2.f")
plt.clim([-10, 10])  # **
plt.show()

### Spectral-centroid

若想要知道這個Spectrogram中頻率大致上在哪邊以及頻率的變化可以計算spectral centroid

$SC[q] = \sum\limits_{k=1}^N{\frac{F[q,k]}{\sum_{k'=1}^N{F[q,k']}}\times k}$

librosa提供```librosa.feature.spectral_centroid```這個功能

In [ ]:
# 使用librosa.feature.spectral_centroid計算頻率中心隨時間的變化
#  'y' or 'S' - 可以選擇由資料y從計算Spectrogram開始，也可以直接丟Spectrogram S 的magnitude
#  'sr' - sampling rate
# 若使用y當input會有以下做STFT的argument可以使用:
#  'n_fft' - FFT點數，同時是window長度，這邊就一定要指定了，預設2048，可根據自己想看到的頻率範圍調整
#  'hop_length' - 每個window間要跳多長
#  'window' - windowing function，預設是 'hann'
# 回傳單位是頻率 Hz
mag_S = abs(googl_S)
googl_sc = librosa.feature.spectral_centroid(S=mag_S, sr=sr)
# N = 2048
# H = 512
# googl_sc = librosa.feature.spectral_centroid(y=googl_,
#                                              sr=sr,
#                                              n_fft=N,
#                                              hop_length=H,
#                                              window='hann' ) # 也可以從訊號開始

In [ ]:
print(googl_S.shape, googl_sc.shape)
print(googl_sc.max(), googl_sc.min())

可以跟spectrogram一起畫，比較著看

當然，因為每個聲音有他的harmonic在裡面會干擾頻率波動的準度 (例如沒有聲音時，整體頻率會掉到比較低的地方)，所以這個centroid還是當參考或者使用centroid來當新的feature提供訓練使用而已。

In [ ]:
times = librosa.times_like(googl_sc)
fig, ax = plt.subplots()
ldp.specshow(googl_S_db, sr=sr, x_axis="s", y_axis="log", cmap='jet')
ax.plot(times, googl_sc.T, label='Spectral centroid', color='k')
ax.legend(loc='upper right')

以上是最常見做頻譜圖的方式。不過就像課堂中講的一樣會有頻率受到window function而有Spectral leakage的問題，並且也有頻率解析度與時間解析度的trade-off，在使用時須多加注意。

In [ ]:
# 這邊試一個Window比較大的情況，會使得頻率的解析度較高，但時間解析度會較差
N = 8192
H = 512

googl_S = librosa.stft(y=googl_, n_fft=N, hop_length=H)
mag_S = abs(googl_S)
googl_sc = librosa.feature.spectral_centroid(S=mag_S, sr=sr)
googl_S_db = librosa.power_to_db(abs(googl_S))  # **
times = librosa.times_like(googl_sc)
fig, ax = plt.subplots()
ldp.specshow(googl_S_db, sr=sr, x_axis="s", y_axis="log", cmap='jet')
ax.plot(times, googl_sc.T, label='Spectral centroid', color='k')
ax.legend(loc='upper right')

## Mel-Spectrogram

在聲音訊號上，為模擬人類聽覺，常使用模擬人耳對頻率變化敏感度的Mel-Spectrogram來做時頻分析。

其中會用到Mel-scale，是Herz經過log轉換後的單位，是因應頻率變化敏感度exponential上升的調整。

<img src='https://i.imgur.com/jgSQKv4.png' width=800>

在做完STFT後，對K'個等長頻段 (在Mel-Scale上等長) 進行triangle filter就是Mel-Spectrogram。

使用的filter bank如下圖所示。

e.g. 10 bins Mel filter bank for 65~1K Hz

<img src='https://i.imgur.com/xDgS9qj.png' width=600>

使用```librosa.feature.melspectrogram```可以針對訊號計算Mel Spectrogram。

In [ ]:
# 使用librosa.feature.melspectrogram計算Mel-Spectrogram
#  'y' or 'S' - 可以選擇由資料y從計算Spectrogram開始，也可以直接丟Spectrogram S 的magnitude
#  'sr' - sampling rate
#  'n_mels' - 要有多少個bin，就是前述的K'
# 若使用y當input會有以下做STFT的argument可以使用:
#  'n_fft' - FFT點數，同時是window長度，這邊就一定要指定了，預設2048，可根據自己想看到的頻率範圍調整
#  'hop_length' - 每個window間要跳多長
#  'window' - windowing function，預設是 'hann'
# 回傳Mel-Spectrum，是實數矩陣，預設為float32的格式
N = 2048
H = 512
K_ = 256  # 這Mel-spectrum的bin數一定會比前面N來得少，才有辦法做filter，不然中間會有很多頻段有缺值
googl_mel_S = librosa.feature.melspectrogram(y=googl_,
                                             sr=sr,
                                             n_mels=K_,
                                             n_fft=N,
                                             hop_length=H,
                                             window='hann')

In [ ]:
# 可以看一些基本性質
print(googl_mel_S.shape)
print(googl_mel_S.min(), googl_mel_S.max())
print(type(googl_mel_S[0, 0]))

在畫spectrogram時記得把y軸scaling換成mel (指的是input單位)，這樣他才會幫忙轉回Hz，並且以Mel scaling排列顯示

librosa設定的mel scale跟log scale會有稍微不同，較專注於較高頻的部分。

In [ ]:
googl_mel_S_db = librosa.power_to_db(googl_mel_S)
plt.figure(figsize=(6, 5))
ldp.specshow(googl_mel_S_db,
             sr=sr,
             x_axis="s",
             y_axis="mel",
             cmap="jet")  # **
plt.colorbar(format="%+4.f")
plt.show()

可以看看到底Mel-Spectrum套用怎樣的filter bank

In [ ]:
mel_banks = librosa.filters.mel(n_fft=2048, sr=sr, n_mels=32)
print(mel_banks.shape)

In [ ]:
plt.figure(figsize=(20, 5))
ldp.specshow(mel_banks.T, sr=sr, y_axis="hz")
plt.colorbar(format="%+2.6f")
plt.xticks(range(32))
plt.show()

如同前面設定的一樣，每個頻段的filter的頻寬都不同，負責越高頻的filter (上圖後面的)頻寬較長。

## Mel-Frequency Cepstral Coefficient

在人聲產生過程中，會經過各種器官的合成，其作用相當於經過一層層的filter。

Mel-Frequency Cepstral Coefficient (MFCC) 來自於地震波的研究，可以用來分離個別filter的響應，或者也可以視為對頻譜進行頻率分析抓出新的feature。

### Cepstrum


Cepstrum就是去做頻譜的頻譜:

$C\{x(t)\}=IFT\{\log(FT\{x(t)\})\}$

<img src="https://i.imgur.com/NqadtG8.png" width=600>

通常在自然訊號中若有週期者，除非是十分完美的sinusoidal波形，在頻譜上都能看得到harmonics。那也就是說這個頻譜上也會具有一定週期性。

Cepstrum的想法就是對log power spectrum進行IFT，這樣就能擷取在頻譜上的週期性特徵。

(稍微說明下，如果對complex spectrum進行IFT則會變回原訊號)

為了做頻譜上的頻率分析，衍伸出了很多相關名詞列在下面，都是一些異體字:

<img src="https://i.imgur.com/8Nna3nn.png" width=600>

### Mel-Frequency Cepstral Coefficient (MFCC)


MFCC是一種Cepstrum (倒頻譜)，是從Mel-Spectrum建立的Cepstrum。
就是在計算完Mel-Spectrum後取log乘以係數轉換成分貝，再做頻域的spectrum。

這邊做spectrum的方式不是做iDFT，而是簡化為discrete cosine transform，沒有虛部，也是因為後來基本比較不看phase頻譜而是magnitude。

DCT: (有各種方式計算，在這邊是舉個範例)
* $X_k=\sum_{N-1}^{n=0}{x_ncos({\pi \over N}(n+{1 \over 2})k)}$

https://en.wikipedia.org/wiki/Discrete_cosine_transform

<img src='https://i.imgur.com/KxS8MSc.png' width=600>

In [ ]:
# librosa.feature.mfcc計算MFCC
#  'y' or 'S' - 可以選擇由資料y從計算Spectrogram開始，也可以直接丟Spectrogram S 的magnitude
#  'sr' - sampling rate
#  'n_mfcc' - 要有多少個bin，就是做iDFT的點數，常用39
# 若使用y當input會有以下做STFT的argument可以使用:
#  'n_fft' - FFT點數，同時是window長度，這邊就一定要指定了，預設2048，可根據自己想看到的頻率範圍調整
#  'hop_length' - 每個window間要跳多長
#  'window' - windowing function，預設是 'hann'
# 回傳Mel-Spectrum，是實數矩陣，預設為float32的格式

googl_mfcc = librosa.feature.mfcc(y=googl_,
                                  sr=sr,
                                  n_mfcc=39,
                                  n_fft=2048,
                                  hop_length=256)

In [ ]:
plt.figure(figsize=(6, 5))
ldp.specshow(googl_mfcc[:, 0:300], x_axis="time", cmap="jet")
plt.colorbar(format="%+4.f")
plt.show()

Cepstrum對頻譜做頻譜，其中第一個rhamonic出現的倒頻就已經代表了harmonics出現在頻譜中的週期，所以通常只看比較前面的倒頻率部分，因此就只有看39個bin而已。

In [ ]:
# 可進一步取MFCC對倒頻率空間的differencing。
googl_delta_mfcc = librosa.feature.delta(googl_mfcc)
plt.figure(figsize=(6, 5))
ldp.specshow(googl_delta_mfcc[:, 0:300], x_axis="time", cmap="jet")
plt.colorbar(format="%+4.f")
plt.show()

以上就是在做ML之前可以做的音訊前處理，處理完之後可以使用DNN、CNN、RNN去做後續分類、迴歸、生成等任務。

## Reference
* M. Mueller, Fundamentals of Music Processing, Springer 2015
* 前述課本附Notebook- https://www.audiolabs-erlangen.de/resources/MIR/FMP/C0/C0.html
* Librosa官網- https://github.com/librosa/librosa
* Acoustics for Musicians and Artists, by Miller Puckette, UCSD
* Youtube 熱門音訊AI課程- https://github.com/musikalkemist/AudioSignalProcessingForML